In [13]:
import random
import math
import numpy as np

import copy


import sys, os, time

from qinfer import LiuWestResampler
from qinfer import utils

from qiskit import IBMQ
from qiskit import QuantumCircuit, execute, Aer
import qiskit.ignis.verification.randomized_benchmarking as rb

IBMQ.load_account()


ibmqfactory.load_account:WARNING:2020-11-14 00:55:09,316: Credentials are already in use. The existing account in the session will be replaced.


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [14]:
import matplotlib.pyplot as plt
import seaborn as sbs

import smc
Distribution = smc.Distribution

sbs.set(style="darkgrid")

%matplotlib inline

In [15]:
def design_circuit(n_qubits, inv_arr, circuit=None):
    
    if circuit is None:
        circuit = QuantumCircuit(n_qubits, n_qubits)
    
    for i, element in enumerate(inv_arr):
        if element == 1:
            circuit.x(i)
    
    circuit.measure(list(range(n_qubits)), list(range(n_qubits)))
    return circuit

In [16]:
def random_circuit(n_qubits):
    length_vector = [1]
    n_seeds = 1
    
    initial_list = list(range(0, n_qubits))    
    rb_pattern = []
    
    while len(initial_list) > 1:
        sample = random.sample(initial_list, 2)
        [initial_list.remove(i) for i in sample]
        rb_pattern.append(sample)
    
    if len(initial_list) == 1:
        rb_pattern.append([initial_list[0]])
    
    
    circuit, xdata = rb.randomized_benchmarking_seq(
        length_vector = [1],
        nseeds = 1,
        rb_pattern = rb_pattern
        )
    circuit = circuit[0][0]
    cregs = circuit.cregs
    
    circuit.remove_final_measurements(inplace=True)
    circuit.cregs = cregs
    
    for i in range(n_qubits):
        circuit.barrier(i)
        if (round(random.random())):
            circuit.x(i)
            circuit.barrier(i)
    
    return circuit

In [17]:
def fake_experiment(true_bias, inversion_arr):
    result = []
    for bias, i in zip(true_bias, inversion_arr):
        outcome = i if random.random() < bias[i] else 1 - i
        result.append(outcome)
    return result

In [20]:
provider = IBMQ.get_provider(group='open', project='main')
backend = provider.get_backend('ibmq_vigo')

n_qubits = 5

n_measurements = 200
n_experiments = 10
n_points = 4000
n_qubits = 5

results = []

for i in range(n_experiments):    

    result_data = {}
    
    circuit = random_circuit(n_qubits)

    result_data['circuit'] = circuit
    result_data['risk'] = []
    result_data['mean'] = []
    result_data['len'] = len(circuit)
    
    dist = Distribution(n_points=n_points, n_qubits=n_qubits)


    for _ in range(n_measurements):
        
        inversion_arr = dist.next_experiment()       
        
        tmp_circuit = copy.deepcopy(circuit)
        tmp_circuit = design_circuit(n_qubits, inversion_arr, circuit=tmp_circuit)

        job = execute(tmp_circuit, backend, shots=1)
        
        result = job.result()
        outcome = list(map(int, list(list(result.get_counts(circuit).keys())[0])))     
        
        print("Measurements: {} Outcome: {}".format(inversion_arr, outcome))
        dist.measure(outcome, inversion_arr)
        
        result_data['risk'].append(dist.calc_bayes_risk())
        result_data['mean'].append(dist.calc_bayes_mean())

    
    result_data['distrubtion'] = dist
    results.append(result_data)


Measurements: [0 0 0 0 0] Outcome: [1, 1, 1, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [0, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 0, 1]
Measurements: [0 0 0 0 0] Outcome: [0, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 0, 1]
Measurements: [1 0 1 1 0] Outcome: [0, 0, 1, 0, 1]
Measurements: [1 0 1 1 0] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 0 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 0 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 0 0 0] Outcome: [0, 1, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 1]
Measurements: [0 1 1 1 1] Outco

Future exception was never retrieved
future: <Future finished exception=ConnectionClosed('WebSocket connection is closed: code = 4002 (private use), no reason')>
websockets.exceptions.ConnectionClosed: WebSocket connection is closed: code = 4002 (private use), no reason


Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 1, 0]


Future exception was never retrieved
future: <Future finished exception=ConnectionClosed('WebSocket connection is closed: code = 4002 (private use), no reason')>
websockets.exceptions.ConnectionClosed: WebSocket connection is closed: code = 4002 (private use), no reason


Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outco

...

Future exception was never retrieved
future: <Future finished exception=ConnectionClosed('WebSocket connection is closed: code = 4002 (private use), no reason')>
websockets.exceptions.ConnectionClosed: WebSocket connection is closed: code = 4002 (private use), no reason


Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 1, 1]


Future exception was never retrieved
future: <Future finished exception=ConnectionClosed('WebSocket connection is closed: code = 4002 (private use), no reason')>
websockets.exceptions.ConnectionClosed: WebSocket connection is closed: code = 4002 (private use), no reason


Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outco

IBMQJobFailureError: 'Unable to retrieve result for job 5fb16e89c824d300139f523f. Job has failed. Use job.error_message() to get more details.'

In [10]:
res_str = '''Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 1, 1]
Measurements: [1 0 0 0 0] Outcome: [0, 0, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 0 1 0 0] Outcome: [0, 0, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 0, 1, 1]
Measurements: [0 0 1 0 0] Outcome: [0, 0, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 0 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 1 0 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 0 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 0 1 1 0] Outcome: [0, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [1 0 1 1 0] Outcome: [0, 1, 0, 1, 1]
Measurements: [0 0 1 0 0] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 1, 1]
Measurements: [0 0 1 0 0] Outcome: [0, 0, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [1 0 1 1 0] Outcome: [0, 1, 0, 1, 1]
Measurements: [1 0 1 0 0] Outcome: [0, 0, 0, 0, 1]
Measurements: [1 0 1 1 0] Outcome: [0, 1, 0, 1, 1]
Measurements: [0 0 1 0 0] Outcome: [1, 0, 0, 1, 0]
Measurements: [0 0 1 0 0] Outcome: [0, 0, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 1, 1]
Measurements: [1 0 0 0 1] Outcome: [0, 0, 1, 1, 1]
Measurements: [1 0 0 0 1] Outcome: [1, 0, 1, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [0, 1, 1, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 0 0 1] Outcome: [1, 0, 1, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [0, 1, 1, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 0, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 0 0 0 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 0 0 1] Outcome: [0, 0, 1, 1, 1]
Measurements: [1 0 0 0 1] Outcome: [1, 0, 1, 0, 1]
Measurements: [0 0 0 0 0] Outcome: [1, 0, 1, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 0 0 0 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 1 0 0 1] Outcome: [0, 1, 0, 1, 0]
Measurements: [0 1 0 0 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 0 0 1] Outcome: [1, 1, 0, 1, 1]
Measurements: [0 0 0 0 1] Outcome: [0, 0, 1, 0, 0]
Measurements: [1 0 0 1 1] Outcome: [1, 1, 0, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 0 0 1 0] Outcome: [1, 0, 0, 1, 0]
Measurements: [0 0 0 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 0 0 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 0 0 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 0 0 1 0] Outcome: [0, 1, 1, 1, 0]
Measurements: [0 0 0 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 0 0 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 0 0 1 0] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 0, 1, 1]
Measurements: [0 0 0 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [1 0 0 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [0 0 0 0 0] Outcome: [0, 1, 1, 0, 0]
Measurements: [0 0 0 1 0] Outcome: [0, 1, 1, 1, 0]
Measurements: [0 0 0 1 0] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 0 1 1] Outcome: [1, 1, 1, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 0, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 1, 0, 0]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 0 0 0] Outcome: [0, 0, 1, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 1, 0]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 0 1 0] Outcome: [1, 1, 1, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [0, 1, 1, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 1, 0, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 1, 0, 0]
Measurements: [1 0 0 1 0] Outcome: [0, 0, 0, 0, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 0, 0, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 0, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 0, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 0, 1, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [1, 0, 1, 0, 0]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 0, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [1, 1, 1, 0, 1]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 0, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [1, 1, 1, 0, 1]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 0, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [1, 0, 0, 1, 0]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 1, 0, 0]
Measurements: [1 0 0 0 0] Outcome: [0, 0, 1, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 0, 0]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 0 0 0] Outcome: [0, 0, 1, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 0, 1, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 1, 0]
Measurements: [1 0 0 1 0] Outcome: [1, 1, 1, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 1, 1, 1]
Measurements: [1 0 0 0 0] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 0 0 0] Outcome: [0, 0, 1, 1, 1]
Measurements: [1 0 0 0 0] Outcome: [0, 0, 1, 1, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 0 1 0 0] Outcome: [0, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 1]
Measurements: [0 1 0 0 0] Outcome: [0, 1, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 1, 1]
Measurements: [0 1 1 0 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 0 0 1] Outcome: [1, 1, 0, 1, 0]
Measurements: [0 1 1 0 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 0 1 0 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 1]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 1, 1]
Measurements: [0 1 1 0 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 1]
Measurements: [0 1 1 0 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 0 1] Outcome: [0, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 0, 1]
Measurements: [0 0 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 0 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 0 1 1 0] Outcome: [0, 1, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 1, 1]
Measurements: [0 0 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 0 1 1 1] Outcome: [1, 0, 0, 1, 0]
Measurements: [0 0 1 1 1] Outcome: [1, 1, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 0 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [1 1 0 0 0] Outcome: [1, 1, 0, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [1, 1, 1, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [0, 1, 1, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [1, 1, 1, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [1, 1, 1, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [1, 1, 1, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [1, 1, 1, 0, 1]
Measurements: [0 0 0 0 0] Outcome: [1, 1, 1, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [1, 1, 0, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [0, 1, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [0 1 0 1 0] Outcome: [1, 0, 1, 1, 0]
Measurements: [0 1 0 1 0] Outcome: [1, 1, 1, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 0 1 0 0] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 0 1] Outcome: [0, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 0, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 0, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 0, 0]
Measurements: [1 0 1 0 0] Outcome: [0, 0, 1, 0, 1]
Measurements: [1 0 1 0 0] Outcome: [0, 0, 0, 0, 1]
Measurements: [1 0 1 1 0] Outcome: [1, 1, 1, 1, 1]
Measurements: [0 0 1 0 0] Outcome: [0, 0, 1, 0, 0]
Measurements: [0 0 1 0 0] Outcome: [1, 0, 1, 1, 1]
Measurements: [0 0 1 0 0] Outcome: [0, 0, 0, 0, 0]
Measurements: [0 0 1 0 0] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 0 0] Outcome: [1, 1, 0, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 0, 1, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 0, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 1, 0, 0]
Measurements: [0 0 0 1 1] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 0 0 1 1] Outcome: [0, 1, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 1, 0, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 0 0 1 1] Outcome: [1, 0, 0, 0, 1]
Measurements: [0 0 0 1 0] Outcome: [0, 0, 0, 1, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 0 0 0 0] Outcome: [1, 0, 0, 1, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 0, 1, 0]
Measurements: [1 0 0 0 0] Outcome: [0, 0, 0, 1, 1]
Measurements: [1 0 0 0 1] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 1 0 1] Outcome: [1, 0, 1, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [0, 1, 0, 1, 1]
Measurements: [1 0 0 0 0] Outcome: [0, 1, 0, 0, 0]
Measurements: [1 0 1 0 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 0, 0, 1]
Measurements: [1 0 0 0 0] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 0 0 0 0] Outcome: [0, 0, 0, 0, 1]
Measurements: [1 0 1 1 0] Outcome: [0, 1, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [0 0 0 0 0] Outcome: [1, 1, 0, 1, 1]
Measurements: [0 0 0 0 0] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 1, 1, 0, 1]
Measurements: [1 0 0 1 0] Outcome: [0, 0, 1, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [0, 0, 0, 0, 1]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 1, 0, 0]
Measurements: [1 0 1 1 0] Outcome: [0, 0, 0, 0, 0]
Measurements: [1 0 1 1 1] Outcome: [1, 0, 0, 0, 0]
Measurements: [1 0 1 1 0] Outcome: [0, 0, 1, 1, 0]
Measurements: [1 0 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [1 0 1 1 0] Outcome: [0, 0, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 0, 1]
Measurements: [0 1 1 1 1] Outcome: [0, 1, 1, 0, 0]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 1]
Measurements: [0 1 1 1 1] Outcome: [1, 0, 1, 1, 1]
'''
measurements_arr = [list(map(int, list(''.join(m.split(' ')[1:6]).replace('[', '').replace(']', '').replace(',', '')))) 
for m in res_str.split('\n')]
outcomes_arr = [list(map(int, list(''.join(m.split(' ')[7:]).replace('[', '').replace(']', '').replace(',', '')))) 
for m in res_str.split('\n')]

In [11]:
n_points = 4000
n_qubits = 5

vals = []
m_outcomes = {}
m_outcomes_order = []

dist = Distribution(n_points=n_points, n_qubits=n_qubits)

for m_number, (measurement, outcome) in enumerate(zip(measurements_arr, outcomes_arr)):    

    dist.measure(outcome, measurement)

    vals.append([dist.calc_bayes_mean(), dist.calc_bayes_risk()])
    
    bayes_mean = dist.calc_bayes_mean()
    m_arr = [1 if bayes_mean[2 * i + 1] < bayes_mean[2 * i] else 0 for i in range(len(bayes_mean) // 2)]
           
    m_arr_str = ''.join(list(map(str, m_arr)))
        
    print(m_arr_str)
    
    if m_number % 10 == 0:
        print("Calculating outcome: ", m_number)
        m_outcomes_order.append(m_arr_str)
        
        if m_arr_str not in m_outcomes:
        
            job = execute(tmp_circuit, backend, shots=4000)

            result = job.result()

            m_outcomes[m_arr_str] = result.get_counts(circuit)    


00111
Calculating outcome:  0


NameError: name 'tmp_circuit' is not defined

In [9]:
ex_res = []
for i in range(0, len(measurements_arr), 15):
    
    inversion_arr = measurements_arr[i]       
        
    tmp_circuit = copy.deepcopy(circuit)
    tmp_circuit = design_circuit(n_qubits, inversion_arr, circuit=tmp_circuit)
    
    job = execute(tmp_circuit, backend, shots=1000)
        
    result = job.result()
    outcome = result.get_counts(circuit)
    
    ex_res.append(outcome)
    print(inversion_arr)
    

NameError: name 'measurements_arr' is not defined

In [169]:
ex_res

[{'00010': 2,
  '10101': 100,
  '10011': 79,
  '11011': 10,
  '10110': 19,
  '00110': 10,
  '10000': 3,
  '01111': 18,
  '01101': 11,
  '11000': 2,
  '01011': 6,
  '10001': 34,
  '00111': 156,
  '01010': 3,
  '11101': 10,
  '10111': 349,
  '01001': 6,
  '11111': 51,
  '00100': 5,
  '10100': 1,
  '11110': 7,
  '01110': 2,
  '11010': 3,
  '01000': 1,
  '10010': 10,
  '11001': 4,
  '00011': 36,
  '11100': 2,
  '00101': 40,
  '00001': 20},
 {'10101': 227,
  '10011': 29,
  '11011': 7,
  '10110': 23,
  '00110': 2,
  '10000': 3,
  '01111': 13,
  '00000': 2,
  '01101': 9,
  '11000': 4,
  '10001': 28,
  '00111': 71,
  '11101': 34,
  '10111': 380,
  '01001': 1,
  '11111': 38,
  '00100': 4,
  '10100': 14,
  '11110': 13,
  '01100': 8,
  '01000': 1,
  '01110': 4,
  '10010': 2,
  '11001': 3,
  '00011': 5,
  '11100': 7,
  '00101': 60,
  '00001': 8},
 {'00010': 1,
  '10101': 237,
  '10011': 39,
  '11011': 5,
  '10110': 22,
  '00110': 5,
  '10000': 2,
  '01111': 8,
  '01101': 10,
  '01011': 3,
  '10001

In [178]:
def bootstrap(result, target=0, samples=900, repeats=1000):
    sampler = []
    for i in result:
        for _ in range(result[i]):
            sampler.append(int(i))
            
    probs = []
    
    for _ in range(repeats):
        tally = 0
        for i in range(samples):
            sample = sampler[random.randint(0, len(sampler) - 1)]
            
            if int(target) == sample:
                tally += 1
        
        probs.append(tally / samples)
    return probs

In [218]:
m_outcomes[m_outcomes_order[2]]

{'00010': 111,
 '10101': 71,
 '10110': 54,
 '00110': 57,
 '10000': 41,
 '01111': 213,
 '11000': 107,
 '00111': 103,
 '01001': 259,
 '00100': 79,
 '10100': 48,
 '11110': 138,
 '01110': 144,
 '11010': 97,
 '10010': 57,
 '11001': 154,
 '00011': 142,
 '11100': 122,
 '00001': 93,
 '10011': 50,
 '11011': 136,
 '00000': 88,
 '01101': 140,
 '01011': 370,
 '10001': 41,
 '01010': 253,
 '11101': 158,
 '10111': 100,
 '11111': 228,
 '01100': 101,
 '01000': 170,
 '00101': 75}

In [185]:
m_outcomes_order

'11111'

In [207]:
n_repeats = 100
x_coords = np.array([[i * 10] * n_repeats for i in range(len(m_outcomes_order))]).flatten()
y_coords = 1 - np.array(
    [bootstrap(
        m_outcomes[m_outcomes_order[i]], 
        target = max(m_outcomes[m_outcomes_order[i]], key=lambda key: m_outcomes[m_outcomes_order[i]][key]), 
        samples=900, repeats=n_repeats) 
     for i in range(len(m_outcomes_order))
    ]).flatten()


In [217]:
y_coords[200]

0.9211111111111111

In [1]:
x_coords_adj = [min(i, 1 - i) for i in x_coords]

ax = sbs.pointplot(x_coords, y_coords, ci=100)

ax.lines[0].set_marker('^')
ax.lines[0].set_linestyle('--')

# london = mlines.Line2D([], [], color='blue', marker='.',
#                           markersize=10, label='IBMQ Virgo')

plt.xlabel("Characterisation Measurements")
plt.ylabel("Error Probability")
plt.title("Randomised Depth 14 Circuit Error Rates (IBMQ Melbourne)")
plt.savefig('figs/error_rates_rb_melbourne.pdf')

NameError: name 'x_coords' is not defined

In [162]:
circuit.draw()

┌───┐             ┌───┐          ┌─────┐┌───┐┌───┐      ░   ┌───┐┌───┐»
qr_0: ─┤ H ├─────────────┤ X ├───────■──┤ SDG ├┤ H ├┤ X ├──────░───┤ X ├┤ H ├»
       ├───┤  ┌───┐   ░  └─┬─┘┌───┐  │  └┬───┬┘└─░─┘└───┘      ░   └───┘└───┘»
qr_1: ─┤ H ├──┤ X ├───░────┼──┤ X ├──┼───┤ H ├───░───────────────────────────»
       ├───┤ ┌┴───┴┐┌───┐  │  └───┘  │   ├───┤   ░        ░        ┌───┐     »
qr_2: ─┤ H ├─┤ SDG ├┤ H ├──┼────■────┼───┤ Z ├────────────░────────┤ Z ├─────»
       ├───┤ └┬───┬┘└───┘  │    │  ┌─┴─┐┌┴───┴┐┌───┐┌───┐ ░    ░   ├───┤┌───┐»
qr_3: ─┤ H ├──┤ S ├────────■────┼──┤ X ├┤ SDG ├┤ H ├┤ Z ├──────░───┤ Z ├┤ H ├»
      ┌┴───┴┐ ├───┤           ┌─┴─┐├───┤└┬───┬┘└───┘└───┘ ░ ┌─────┐├───┤└───┘»
qr_4: ┤ SDG ├─┤ H ├───────────┤ X ├┤ H ├─┤ S ├────────────░─┤ SDG ├┤ H ├─────»
      └─────┘ └───┘           └───┘└───┘ └───┘            ░ └─────┘└───┘     »
cr_0: ═══════════════════════════════════════════════════════════════════════»
                                                                             »
cr_1: ═══════════════════════════════════════════════════════════════════════»
                                                                             »
cr_2: ═══════════════════════════════════════════════════════════════════════»
                                                                             »
cr_3: ═══════════════════════════════════════════════════════════════════════»
                                                                             »
cr_4: ═══════════════════════════════════════════════════════════════════════»
                                                                             »
«      ┌───┐               ┌───┐ ┌───┐   ░  ┌───┐  ░     
«qr_0: ┤ S ├───────■───────┤ X ├─┤ H ├───░──┤ X ├──░─────
«      └───┘       │       └─┬─┘ └───┘   ░  └───┘  ░     
«qr_1: ────────────┼─────────┼───────────────────────────
«           ┌───┐  │  ┌───┐  │   ┌───┐   ░               
«qr_2: ──■──┤ H ├──┼──┤ S ├──┼───┤ H ├───░───────────────
«        │  ├───┤┌─┴─┐└───┘  │  ┌┴───┴┐┌───┐  ░  ┌───┐ ░ 
«qr_3: ──┼──┤ S ├┤ X ├───────■──┤ SDG ├┤ H ├──░──┤ X ├─░─
«      ┌─┴─┐├───┤├───┤  ░       └─────┘└───┘  ░  └───┘ ░ 
«qr_4: ┤ X ├┤ H ├┤ S ├──░────────────────────────────────
«      └───┘└───┘└───┘  ░                                
«cr_0: ══════════════════════════════════════════════════
«                                                        
«cr_1: ══════════════════════════════════════════════════
«                                                        
«cr_2: ══════════════════════════════════════════════════
«                                                        
«cr_3: ══════════════════════════════════════════════════
«                                                        
«cr_4: ══════════════════════════════════════════════════
«

In [196]:
s = '''00111
Calculating outcome:  0
11111
00011
01010
00011
11011
10001
00000
11001
10001
00001
Calculating outcome:  10
00011
00011
00001
00001
00011
00010
00011
00010
00010
00010
Calculating outcome:  20
00010
00010
00010
00011
00011
00011
00011
00011
00011
00011
Calculating outcome:  30
00011
00011
00011
00011
00011
00011
00011
00011
00011
00011
Calculating outcome:  40
00011
00011
00011
00011
00011
10011
00011
00011
00011
00011
Calculating outcome:  50
00011
00011
10011
10011
10011
10011
10011
10011
10011
10011
Calculating outcome:  60
10011
10011
10110
10110
10110
10111
10111
10111
10011
10011
Calculating outcome:  70
10011
10011
10011
10011
10011
10011
10011
10011
10011
10011
Calculating outcome:  80
10011
10011
10011
10011
10011
10010
10011
10011
10011
10011
Calculating outcome:  90
10011
10011
10011
10011
10011
10011
10011
10011
10011
10011
Calculating outcome:  100
10001
10011
10011
10010
10001
10001
10001
10001
10001
10101
Calculating outcome:  110
10101
10101
10101
00111
00111
10101
10101
10101
10111
10111
Calculating outcome:  120
10111
00110
00111
00111
00110
00110
00100
10100
10100
10100
Calculating outcome:  130
10100
10100
10100
10100
10100
10100
10100
10100
00100
00100
Calculating outcome:  140
00100
00100
00100
10100
10100
10100
00100
00100
00100
00101
Calculating outcome:  150
00100
10100
10100
00100
10100
10100
00100
00100
00100
00100
Calculating outcome:  160
00100
00100
00100
00100
00100
00100
00100
00100
00100
00100
Calculating outcome:  170
01100
00100
00100
00100
00100
01100
00100
00100
01100
01100
Calculating outcome:  180
01100
01100
01100
01100
01100
01100
01100
01100
01100
01100
Calculating outcome:  190
01100
01100
01100
01100
01100
01100
01100
01100
01100
01100
Calculating outcome:  200
01100
01100
01100
01100
01100
01100
01100
01100
01100
01100
Calculating outcome:  210
01100
01100
01100
01100
01100
01100
01110
01100
01110
01110
Calculating outcome:  220
01110
01110
01110
00110
00110
00110
00110
00110
00110
00110
Calculating outcome:  230
00110
00110
00110
00110
00110
00100
00110
00110
00110
00100
Calculating outcome:  240
00100
00100
00100
00100
00100
00100
00100
00100
00100
00100
Calculating outcome:  250
00100
01100
01100
01100
00100
00100
00100
00100
00100
00100
Calculating outcome:  260
00100
00100
00100
00100
00100
00100
00100
00100
00100
00100
Calculating outcome:  270
00100
00100
00100
00100
00100
00100
00100
00100
00100
00100
Calculating outcome:  280
00100
00100
00100
00100
00100
00100
00100
00100
00100
00100
Calculating outcome:  290
00100
00100
00100
00100
00100
00100
00100
00100
00100
00100
Calculating outcome:  300
00100
00100
00100
00100
00100
00100
00100
00100
00100
00100
Calculating outcome:  310
00100
00100
00100
00100
00100
00100
00100
00100
00100
00100
Calculating outcome:  320
00100
00100
00100
00100
00100
00100
00100
00100
00100
00100
Calculating outcome:  330
00100
00100
00100
00100
00100
00100
00100
00100
00100
00100
Calculating outcome:  340
00100
00100
00100
00100
00100
00100
00100
00100
00100
00100
Calculating outcome:  350
00100
00100
00100
00100
00100
00100
00100
00100
00100
00100
Calculating outcome:  360
00100
00100
00100
00100
00100
00100
00100
00100
00100
00100
Calculating outcome:  370
00100
00100
00100
00100
00100
00100
00100
00100
00100
00100
Calculating outcome:  380
00100
00100
00100
00100
00100
00100
00100
00100
00100
00100
Calculating outcome:  390
00100
00100
00100
00100
00100
00100
00100
00100
00100
00100
Calculating outcome:  400
00100
00100
00100
00100
00100
00100
00100
00100
00100
00100
Calculating outcome:  410
00100
00100
00100
00100
00100
00100
00100
00100
00100
00100
Calculating outcome:  420
00100
00100
00100
00100
10100
00100
00100
00100
00100
00100
Calculating outcome:  430
00100
00100
00100
00100
00100
00100
00100
00100
00100
00100
Calculating outcome:  440
00100
00100
00100
00100
00100
00100
10100
00100
00100
00100
Calculating outcome:  450
10100
10100
10100
10100
10100
10100
10100
10100
10100
10100
Calculating outcome:  460
10100
10100
10100
10100
10100
10100
10100
10100
10100
10100
Calculating outcome:  470
10100
10100
11100
11100
11100
11100
11100
11100
11100
11100
Calculating outcome:  480
11100
11100
11100
11100
11100
01100
01100
01100
01100
01100
Calculating outcome:  490
01100
01100
01100
01100
01100
01100
01100
01100
01100
01100
Calculating outcome:  500
01100
01100
01100
01100
01100
01100
01100
01100
01000
01000
Calculating outcome:  510
01000
01000
01000
01000
01000
01000
01000
01000
01000
01000
Calculating outcome:  520
01000
01000
01000
01000
01100
01100
01100
01100
01100
01100
Calculating outcome:  530
01100
11100
11101
11101
11101
11101
11101
11101
11101
11101
Calculating outcome:  540
11101
11101
11101
11101
11101
11101
11101
11101
11101
11101
Calculating outcome:  550
11101
11101
11101
11101
11101
11101
11101
11111
11111
11111
Calculating outcome:  560'''

In [197]:
s = s.split('\n')
order = []
for i in range(len(s)):
    if s[i][0] == 'C':
        order.append(s[i - 1])